<a href="https://colab.research.google.com/github/KumarGaurav20/Spam-Emails/blob/main/TRAINING_NAIVE_BAYES_CLASSIFIER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TRAINING NAIVE BAYES CLASSIFIER

IMPORTS

CONSTANTS

ACESSING DATA


FUNC FOR CONVERSION OF SPARSE MATRIX TO FULL MATRIX